In [2]:
import twarc
import os
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
import requests
from transformers import pipeline
import string
from transformers import pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

## Process the downloaded tweet json to csv files

In [5]:
all_columns=['conversation_id', 'id','author_id','text','in_reply_to_user_id']
for file in os.listdir("fix"):
    if file.split(".")[1]=="json":
        print("--------------Processing file {0}--------------".format(file))
        df=pd.read_json(os.path.join("fix",file),lines=True)
        all_tweets=[]
        i=100
        for line in df['data']:
            for tweet in line:
                tweet_info=[]

                for column in all_columns:
                    tweet_info.append(tweet[column])

                if tweet.get('referenced_tweets','')!='':
                    if tweet['referenced_tweets'][0]['type']=='replied_to':
                        tweet_info.append(tweet['referenced_tweets'][0]['id'])
                    else:
                        tweet_info.append('')
                else:
                    tweet_info.append('')
                
                if tweet.get('attachments','')!='':
                    tweet_info.append('1')
                else:
                    tweet_info.append('0')

                all_tweets.append(tweet_info)

            if len(all_tweets)>i:
                print("--------------Processed more than {0} tweets--------------".format(i))
                i=i*10

        for line in df['includes']:
            for tweet in line['tweets']:
                tweet_info=[]

                for column in all_columns:
                    tweet_info.append(tweet.get(column,''))

                if tweet.get('referenced_tweets','')!='':
                    if tweet['referenced_tweets'][0]['type']=='replied_to':
                        tweet_info.append(tweet['referenced_tweets'][0]['id'])
                    else:
                        tweet_info.append('')
                else:
                    tweet_info.append('')
                
                if tweet.get('attachments','')!='':
                    tweet_info.append('1')
                else:
                    tweet_info.append('0')
                
                all_tweets.append(tweet_info)

            if len(all_tweets)>i:
                print("--------------Processed more than {0} tweets--------------".format(i))
                i=i*10

        df_out = pd.DataFrame(all_tweets,columns=all_columns+['replied_to', 'attachment'])
        df_out = df_out.drop_duplicates().copy()
        df_out.to_csv("fix_output/"+file.split(".")[0]+".csv", index=False)

--------------Processing file New_Britain_ids.json--------------
--------------Processed more than 100 tweets--------------
--------------Processed more than 1000 tweets--------------
--------------Processed more than 10000 tweets--------------
--------------Processed more than 100000 tweets--------------
--------------Processing file New_nepal_ids.json--------------
--------------Processed more than 100 tweets--------------
--------------Processed more than 1000 tweets--------------
--------------Processing file New_Australia_ids.json--------------
--------------Processed more than 100 tweets--------------
--------------Processed more than 1000 tweets--------------
--------------Processed more than 10000 tweets--------------
--------------Processing file Ireland_ids.json--------------
--------------Processed more than 100 tweets--------------
--------------Processed more than 1000 tweets--------------
--------------Processing file boyscouts.json--------------
--------------Processed m

In [6]:
#Associate files for right namings
ivan_dict={"nepal_ids":"NepalRedCross","Nairobi_ids":"ICRC_Nairobi","Canada_ids":"redcrosscanada","Britain_ids":"BritishRedCross","Australia_ids":"RedCrossAU","Ireland_ids":"irishredcross"}

In [9]:
#Concatenate files from different jsons
NGO_list = ["Australia_ids", "Britain_ids", "Canada_ids", "Ireland_ids", "Nairobi_ids", "nepal_ids", "RedCrossLebanon", "RedCross", "philredcross", "IFRC_Europe", "ICRC_library"]
files=os.listdir("fix_output")
for ngo in NGO_list:
    group=[file for file in files if ngo in file]
    if 'RedCross.csv' in group:
        group=[element for element in group if 'Lebanon' not in element]
    for ind,element in enumerate(group):
        if ind==0:
            df=pd.read_csv("fix_output/"+element, dtype='str')
        else:
            df1=pd.read_csv("fix_output/"+element, dtype='str')
            df=pd.concat([df,df1]).copy()
    df.to_csv("fix_output_all/"+ivan_dict.get(ngo,ngo)+"_all.csv",index=False)
        

## Perform labeling for answered/unanswered tweets

In [48]:
#Dictionary to store NGOs ids
NGO_dict={"RedCrossAU":"138418895",
        "BritishRedCross":"7400692",
        "philredcross":"32391821",
        "redcrosscanada":"16434613",
        "ICRC_Nairobi":"1151791608494473219",
        "NepalRedCross":"566269074",
        "RedCrossLebanon":"2548704956",
        "ICRC_library":"2548704956",
        "RedCross":"6519522",
        "irishredcross":"25411906",
        "IFRC_Europe":"937963107456036865",
        "Ch_JesusChrist":"10047382",
        "ymca":"309679834",
        "SalvationArmyUS":"16729099",
        "boyscouts":"20685982",
        "girlscouts":"103018203",
        "Habitat_org":"33898911",
        "SavetheChildren":"14055301",
        "WorldVision":"14086764",
        "RESCUEorg":"22053725",
        "CatholicRelief":"14496886"}

In [49]:
def do_labeling(org_ids):

    #DataFrame to store all labeled replies related to our task
    all_replies = pd.DataFrame()

    for file in os.listdir('fix_output_all'):
        if file.split('.')[1]=='csv': #set '_all. for ICRC'
            NGO_name = file.split('.')[0] #set '_all.' for ICRC
            NGO_id = org_ids[NGO_name]

            #added terminator to fix tokenizer error for some files
            df = pd.read_csv("fix_output_all/{0}".format(file), lineterminator='\n', dtype='str')

            #get the tweets on which users got replies
            ngo_replied_to_ids = df[(df["author_id"] == NGO_id) & (df["in_reply_to_user_id"] != NGO_id)]["replied_to"].dropna()
            ngo_replied_to = df[df['id'].isin(ngo_replied_to_ids)].dropna()
            ngo_replied_to["label"] = 1

            #get the tweets on which ngo didn't reply
            replies_to_ngo = df[(df["in_reply_to_user_id"]==NGO_id) & (df["author_id"]!=NGO_id)]
            replies_to_ngo = replies_to_ngo[~replies_to_ngo['id'].isin(ngo_replied_to_ids)].dropna()
            replies_to_ngo["label"] = 0

            #concatenate these tweets together
            all_ngo_replies = pd.concat([replies_to_ngo, ngo_replied_to]).reset_index(drop=True)
            all_ngo_replies["relatedNGO"] = NGO_name
            
            #add current ngo data to global df
            all_replies = pd.concat([all_replies, all_ngo_replies]).reset_index(drop=True)

            print("processed file: {0}".format(file))
            
    all_replies.to_csv("fix_labeled/labeled_data.csv")


In [50]:
#Label NGO
do_labeling(NGO_dict)

processed file: Habitat_org.csv
processed file: RedCross.csv
processed file: SavetheChildren.csv
processed file: boyscouts.csv
processed file: BritishRedCross.csv
processed file: SalvationArmyUS.csv
processed file: ICRC_library.csv
processed file: redcrosscanada.csv
processed file: ICRC_Nairobi.csv
processed file: philredcross.csv
processed file: Ch_JesusChrist.csv
processed file: NepalRedCross.csv
processed file: CatholicRelief.csv
processed file: RESCUEorg.csv
processed file: RedCrossAU.csv
processed file: irishredcross.csv
processed file: RedCrossLebanon.csv
processed file: ymca.csv
processed file: girlscouts.csv
processed file: IFRC_Europe.csv
processed file: WorldVision.csv


In [110]:
df = pd.read_csv("fix_labeled/labeled_data.csv", index_col=0, dtype='str')
df.shape

(116416, 9)

## Process downloaded authors info

In [ ]:
#Process the needed info to csv
df=pd.read_json("fix_labeled/users.json",lines=True)
all_users=[]
public_columns=['followers_count', 'following_count','tweet_count']
additional_columns=['id','name', 'username', 'description','created_at']
for line in df['data']:
    for user in line:

        user_info=[]        
        for column in public_columns:
            try:
                user_info.append(user['public_metrics'][column])
            except:
                print(user)
        for column in additional_columns:
            user_info.append(user[column])

        all_users.append(user_info)

users = pd.DataFrame(all_users,columns=public_columns+additional_columns)
users = users.drop_duplicates().copy()
users = users.drop_duplicates(["id"]).copy()
users.to_csv("fix_labeled/users.csv", index=False)

In [113]:
#read df
users =  pd.read_csv("users/users.csv", dtype='str')
users

,followers_count,following_count,tweet_count,id,name,username,description,created_at
0,1505,193,19932,19279287,Mobile Home Gurl,mobilehomegurl,"Best-Selling Author, Real Estate Investor, & S...",2009-01-21T06:51:36.000Z
1,1316,2565,186281,1461435318574493712,Joan,joans3627,🌹💜🐈🦮📚🧑‍🍳🥾🌏🪐,2021-11-18T20:45:38.000Z
2,69,231,2090,889333376,Phillips Flooring America,Phillips_Floor,We're a full-service retail flooring sales and...,2012-10-18T17:21:44.000Z
3,290,556,495,3293184712,"Janeal M. White, PhD, CFLE, MIE",janealmwhite,Emerging scholar. Passionate educator. World c...,2015-05-21T15:47:55.000Z
4,1142,4824,3067,3821162059,Marie LeBlanc,humanendangered,Disability Artist/Advocate \n\n#overdressed\n#...,2015-10-08T03:29:53.000Z
...,...,...,...,...,...,...,...,...
74752,1831,758,24313,210373839,escapism,fastfoodbjork,I miss my sister. she's such a bitch.\nBuildin...,2010-10-31T08:43:56.000Z
74753,5924,4356,46080,740637548,Freda Ou 1,FredaOu,当魔鬼吼叫得越来越慘历的时候，黎明已越来越近了........！ 光复香港，时代革命！ ...,2012-08-06T13:35:02.000Z
74754,3473,3968,128603,701322706495602688,Tania Groth (Gettr: Tania Groth - @Falkoner1),TaniaGroth,Conservative Capitalist\nONLY 2 genders\nLoath...,2016-02-21T08:28:57.000Z
74755,498,1060,34326,312255461,Kashmir isnt India!!,ipeace4all,NaN,2011-06-06T20:41:34.000Z


In [124]:
#merge with existing tweet df
df_merged = df.merge(users, left_on='author_id', right_on='id', how='left')
df_merged = df_merged.drop(['name', 'username', 'description', 'created_at', 'id_y'], axis = 1)
df_merged = df_merged.rename(columns={"id_x":"id", "followers_count":"author_followers", "following_count":"author_following", "tweet_count":"author_tweets"})
df_merged

,conversation_id,id,author_id,text,in_reply_to_user_id,replied_to,attachment,label,relatedNGO,author_followers,author_following,author_tweets
0,128892368731643905,129039727990026241,19279287,RT @Habitat_org For anyone in the Joplin area ...,33898911,128892368731643905,0,0,Habitat_org,1505,193,19932
1,1513883133417979904,1514283971437383683,1461435318574493712,@Habitat_org @MrDrewScott @JonathanScott 👏👏👏👏👏,33898911,1513883133417979904,0,0,Habitat_org,1316,2565,186281
2,1150423207129886720,1150806881885130752,889333376,@Habitat_org @HabitatPL 👍🙂,33898911,1150423207129886720,0,0,Habitat_org,69,231,2090
3,1150423207129886720,1150469798335266817,3293184712,@Habitat_org @HabitatPL So thankful for Habita...,33898911,1150423207129886720,0,0,Habitat_org,290,556,495
4,1370041829408784384,1370457258673401860,3821162059,@Habitat_org Wow! This would be awesome for th...,33898911,1370041829408784384,0,0,Habitat_org,1142,4824,3067
...,...,...,...,...,...,...,...,...,...,...,...,...
116411,1093570142888435713,1093570501375676416,474681580,"@WorldVision You go girl💃💃💃,there's alot of pe...",14086764,1093570142888435713,0,1,WorldVision,725,922,7881
116412,1062866805545492481,1063045758188969984,958755194212405248,@ericmetaxas @obianuju @WorldVision @PatriciaH...,105901883,1062866805545492481,0,1,WorldVision,406,1478,56632
116413,931869631702331392,931881262201102336,928471317535514624,@WorldVision Arthur's Song Charity's is always...,14086764,931869631702331392,0,1,WorldVision,177,2086,497
116414,1067115815001096193,1067207873061666821,196878226,@WorldVision 1 Peter 5:7 Cast all your anxiet...,14086764,1067115815001096193,0,1,WorldVision,12,333,274


In [125]:
#Save the data
df_merged.to_csv("fix_labeled/labeled_users_merged.csv")

## Process downloaded info of users in reply to whom the tweet was written

In [153]:
#Process the needed info to csv
df=pd.read_json("fix_labeled/users.json",lines=True)
all_users=[]
public_columns=['followers_count', 'following_count','tweet_count']
additional_columns=['id','name', 'username', 'description','created_at']
for line in df['data']:
    for user in line:

        user_info=[]        
        for column in public_columns:
            try:
                user_info.append(user['public_metrics'][column])
            except:
                print(user)
        for column in additional_columns:
            user_info.append(user[column])

        all_users.append(user_info)

df_out = pd.DataFrame(all_users,columns=public_columns+additional_columns)
df_out = df_out.drop_duplicates().copy()
df_out.to_csv("fix_labeled/replied_users.csv", index=False)

In [154]:
#read df
replied_to = pd.read_csv("fix_labeled/replied_users.csv", dtype='str')
replied_to

,followers_count,following_count,tweet_count,id,name,username,description,created_at
0,1018483,5298,28947,33898911,Habitat_org,Habitat_org,Help build a world where everyone has a decent...,2009-04-21T13:16:09.000Z
1,554,1227,1497,39549725,Andrea Higgins,writeinatlanta,Revolutionary people and extraordinary events ...,2009-05-12T17:35:45.000Z
2,3587,137,2928,594917351,Jonathan Reckford,JReckford,"Husband, father & big believer in the power of...",2012-05-30T21:21:06.000Z
3,29513,8637,49663,1080590139829608448,pantslessandplanless,vodkanopants,Clinically hysterical disabled artist travels ...,2019-01-02T22:22:20.000Z
4,13,159,344,1279878726885482498,Katy Lynne 🙈🙉🙊,Katydid527,NaN,2020-07-05T20:44:11.000Z
...,...,...,...,...,...,...,...,...
1462,9174,2524,5968,291765618,Vision du Monde,VisionduMondeFR,"Membre de @WorldVision, 1ère ONG de parrainage...",2011-05-02T15:44:12.000Z
1463,24537,44,48217,873640802,Star Dünya,StarDisHaberler,https://t.co/1cskzzjpc3 - Dünya'dan haberler,2012-10-11T13:38:18.000Z
1464,2448,639,4101,1469799104,World Vision MEERO,WVMEERO,World Vision in the Middle East and Eastern Eu...,2013-05-30T13:52:44.000Z
1465,0,11,226,864233580309864448,Connie Penrod,penrod_connie,NaN,2017-05-15T21:38:37.000Z


In [161]:
#merge with existing tweet df
df_merged = df_merged.merge(replied_to, left_on='in_reply_to_user_id', right_on='id', how='left')
df_merged = df_merged.drop(['name', 'description', 'created_at', 'id_y'], axis = 1)
df_merged = df_merged.rename(columns={"id_x":"id", "followers_count":"replied_followers", "following_count":"replied_following", "tweet_count":"replied_tweets", "username":"replied_name"})
df_merged

,conversation_id,id,author_id,text,in_reply_to_user_id,replied_to,attachment,label,relatedNGO,author_followers,author_following,author_tweets,replied_followers,replied_following,replied_tweets,replied_name
0,128892368731643905,129039727990026241,19279287,RT @Habitat_org For anyone in the Joplin area ...,33898911,128892368731643905,0,0,Habitat_org,1505,193,19932,1018483,5298,28947,Habitat_org
1,1513883133417979904,1514283971437383683,1461435318574493712,@Habitat_org @MrDrewScott @JonathanScott 👏👏👏👏👏,33898911,1513883133417979904,0,0,Habitat_org,1316,2565,186281,1018483,5298,28947,Habitat_org
2,1150423207129886720,1150806881885130752,889333376,@Habitat_org @HabitatPL 👍🙂,33898911,1150423207129886720,0,0,Habitat_org,69,231,2090,1018483,5298,28947,Habitat_org
3,1150423207129886720,1150469798335266817,3293184712,@Habitat_org @HabitatPL So thankful for Habita...,33898911,1150423207129886720,0,0,Habitat_org,290,556,495,1018483,5298,28947,Habitat_org
4,1370041829408784384,1370457258673401860,3821162059,@Habitat_org Wow! This would be awesome for th...,33898911,1370041829408784384,0,0,Habitat_org,1142,4824,3067,1018483,5298,28947,Habitat_org
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116411,1093570142888435713,1093570501375676416,474681580,"@WorldVision You go girl💃💃💃,there's alot of pe...",14086764,1093570142888435713,0,1,WorldVision,725,922,7881,1092933,31529,13934,WorldVision
116412,1062866805545492481,1063045758188969984,958755194212405248,@ericmetaxas @obianuju @WorldVision @PatriciaH...,105901883,1062866805545492481,0,1,WorldVision,406,1478,56632,179716,764,69663,ericmetaxas
116413,931869631702331392,931881262201102336,928471317535514624,@WorldVision Arthur's Song Charity's is always...,14086764,931869631702331392,0,1,WorldVision,177,2086,497,1092933,31529,13934,WorldVision
116414,1067115815001096193,1067207873061666821,196878226,@WorldVision 1 Peter 5:7 Cast all your anxiet...,14086764,1067115815001096193,0,1,WorldVision,12,333,274,1092933,31529,13934,WorldVision


In [163]:
#Save the data
df_merged.to_csv("fix_labeled/labeled_users_merged.csv")

## Add additional features

In [122]:
df = pd.read_csv("fix_labeled/labeled_users_merged.csv", dtype='str', index_col= 0)
df.head(5)

,conversation_id,id,author_id,text,in_reply_to_user_id,replied_to,attachment,label,relatedNGO,author_followers,author_following,author_tweets,replied_followers,replied_following,replied_tweets,replied_name
0,128892368731643905,129039727990026241,19279287,RT @Habitat_org For anyone in the Joplin area ...,33898911,128892368731643905,0,0,Habitat_org,1505,193,19932,1018483,5298,28947,Habitat_org
1,1513883133417979904,1514283971437383683,1461435318574493712,@Habitat_org @MrDrewScott @JonathanScott 👏👏👏👏👏,33898911,1513883133417979904,0,0,Habitat_org,1316,2565,186281,1018483,5298,28947,Habitat_org
2,1150423207129886720,1150806881885130752,889333376,@Habitat_org @HabitatPL 👍🙂,33898911,1150423207129886720,0,0,Habitat_org,69,231,2090,1018483,5298,28947,Habitat_org
3,1150423207129886720,1150469798335266817,3293184712,@Habitat_org @HabitatPL So thankful for Habita...,33898911,1150423207129886720,0,0,Habitat_org,290,556,495,1018483,5298,28947,Habitat_org
4,1370041829408784384,1370457258673401860,3821162059,@Habitat_org Wow! This would be awesome for th...,33898911,1370041829408784384,0,0,Habitat_org,1142,4824,3067,1018483,5298,28947,Habitat_org


In [ ]:
#drop rows with uncorrectly processed label if such exist
df = df[df['label'].notna()]
df = df.reset_index(drop=True)

In [ ]:
#prepare for text tokenization
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
stop_words.append('uh')
puncs = string.punctuation

In [126]:
#tokenize the text
df["text_new"]=df["text"].apply(lambda x: [word for word in [re.sub(f"[{puncs}]","",token).lower() for token in x.split() if token[0]!='@'] if word not in stop_words+['rt','']])
#number of user mentions "@" in a text
df["num_mentions"]=df["text"].apply(lambda x: len([i for i in x.split() if i[0]=='@']))
#number of links in a text
df["num_links"]=df["text"].apply(lambda x: len([i for i in x.split() if 'http'in i]))
#number of words in original text
df["num_full_words"] = df.text.apply(lambda x: len(x.split(' ')))
#number of words in tokenized text
df["num_tokenized_words"] = df.text_new.apply(lambda x: len(x))
#number of hashtags in text
df["num_hashtags"]=df.text.apply(lambda x: len([i for i in x.split() if i[0]=='#']))
#number of related_NGO mentions in a tweet
df["num_NGO_mentions"]=df.apply(lambda x: x.text.split(" ").count(f"@{x.relatedNGO}"), axis=1)
#number of exclamation marks in text
df["num_exclamation"]=df.apply(lambda x: x.text.count("!"), axis=1)
#number of question marks in a tweet
df["num_question"]=df.apply(lambda x: x.text.count("?"), axis=1)
#if the tweet was a retweeted one
df["retweet"] = df.text.apply(lambda x: 1 if x[:2]=="RT" else 0)
#number of characters in original text
df["num_characters"] = df.text.apply(lambda x: len(x))

df

,conversation_id,id,author_id,text,in_reply_to_user_id,replied_to,attachment,label,relatedNGO,author_followers,...,replied_name,text_new,num_mentions,num_links,num_full_words,num_tokenized_words,num_hashtags,num_NGO_mentions,num_exclamation,num_question
0,128892368731643905,129039727990026241,19279287,RT @Habitat_org For anyone in the Joplin area ...,33898911,128892368731643905,0,0,Habitat_org,1505,...,Habitat_org,"[anyone, joplin, area, habitat, humanity, need...",2,0,20,11,0,1,0,0
1,1513883133417979904,1514283971437383683,1461435318574493712,@Habitat_org @MrDrewScott @JonathanScott 👏👏👏👏👏,33898911,1513883133417979904,0,0,Habitat_org,1316,...,Habitat_org,[👏👏👏👏👏],3,0,4,1,0,1,0,0
2,1150423207129886720,1150806881885130752,889333376,@Habitat_org @HabitatPL 👍🙂,33898911,1150423207129886720,0,0,Habitat_org,69,...,Habitat_org,[👍🙂],2,0,3,1,0,1,0,0
3,1150423207129886720,1150469798335266817,3293184712,@Habitat_org @HabitatPL So thankful for Habita...,33898911,1150423207129886720,0,0,Habitat_org,290,...,Habitat_org,"[thankful, habitat, partners, working, elimina...",2,0,29,16,0,1,0,0
4,1370041829408784384,1370457258673401860,3821162059,@Habitat_org Wow! This would be awesome for th...,33898911,1370041829408784384,0,0,Habitat_org,1142,...,Habitat_org,"[wow, would, awesome, us, living, multiple, ch...",1,0,18,9,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116411,1093570142888435713,1093570501375676416,474681580,"@WorldVision You go girl💃💃💃,there's alot of pe...",14086764,1093570142888435713,0,1,WorldVision,725,...,WorldVision,"[go, girl💃💃💃theres, alot, people, rooting]",1,0,10,5,0,1,2,0
116412,1062866805545492481,1063045758188969984,958755194212405248,@ericmetaxas @obianuju @WorldVision @PatriciaH...,105901883,1062866805545492481,0,1,WorldVision,406,...,ericmetaxas,"[wow, ive, lost, total, respect, world, vision...",4,0,13,8,0,1,2,0
116413,931869631702331392,931881262201102336,928471317535514624,@WorldVision Arthur's Song Charity's is always...,14086764,931869631702331392,0,1,WorldVision,177,...,WorldVision,"[arthurs, song, charitys, always, worlds, peop...",1,0,21,10,0,1,0,0
116414,1067115815001096193,1067207873061666821,196878226,@WorldVision 1 Peter 5:7 Cast all your anxiet...,14086764,1067115815001096193,0,1,WorldVision,12,...,WorldVision,"[1, peter, 57, cast, anxiety, cares]",1,0,13,6,0,1,0,0


In [105]:
#save the data
df.to_csv("fix_labeled/extended_features.csv")

## Sentiment analysis of tweets

In [88]:
#set hugging face model
model = "cardiffnlp/twitter-roberta-base-sentiment-latest"

#set the pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model = model, tokenizer = model)

#check output for one of the tweets
sentiment_pipeline(df["text"][0])[0]["label"]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


'neutral'

In [80]:
#check tweet text
df["text"][0]

'RT @Habitat_org For anyone in the Joplin area RT @joplinglobe: Habitat for Humanity needs volunteers to haul building supplies donated...'

In [111]:
#get sentiment while treating errors
def set_sentimet(text):
    try:
        sentiment = sentiment_pipeline(text)[0]["label"]
    except:
        sentiment = "Error"
    return sentiment

In [114]:
#add sentiment feature to the dataset
df["sentiment"] = df.text.apply(lambda x: set_sentimet(x))
df = df[df.sentiment!="Error"]
df

,conversation_id,id,author_id,text,in_reply_to_user_id,replied_to,attachment,label,relatedNGO,author_followers,...,num_mentions,num_links,num_full_words,num_tokenized_words,num_hashtags,num_NGO_mentions,num_exclamation,num_question,has_question,sentiment
0,128892368731643905,129039727990026241,19279287,RT @Habitat_org For anyone in the Joplin area ...,33898911,1.288924e+17,0.0,0.0,Habitat_org,1505.0,...,2,0,20,11,0,1,0,0,0,neutral
1,1513883133417979904,1514283971437383683,1461435318574493712,@Habitat_org @MrDrewScott @JonathanScott 👏👏👏👏👏,33898911,1.513883e+18,0.0,0.0,Habitat_org,1316.0,...,3,0,4,1,0,1,0,0,0,positive
2,1150423207129886720,1150806881885130752,889333376,@Habitat_org @HabitatPL 👍🙂,33898911,1.150423e+18,0.0,0.0,Habitat_org,69.0,...,2,0,3,1,0,1,0,0,0,positive
3,1150423207129886720,1150469798335266817,3293184712,@Habitat_org @HabitatPL So thankful for Habita...,33898911,1.150423e+18,0.0,0.0,Habitat_org,290.0,...,2,0,29,16,0,1,0,0,0,positive
4,1370041829408784384,1370457258673401860,3821162059,@Habitat_org Wow! This would be awesome for th...,33898911,1.370042e+18,0.0,0.0,Habitat_org,1142.0,...,1,0,18,9,0,1,1,0,0,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116411,1093570142888435713,1093570501375676416,474681580,"@WorldVision You go girl💃💃💃,there's alot of pe...",14086764,1.093570e+18,0.0,1.0,WorldVision,725.0,...,1,0,10,5,0,1,2,0,0,positive
116412,1062866805545492481,1063045758188969984,958755194212405248,@ericmetaxas @obianuju @WorldVision @PatriciaH...,105901883,1.062867e+18,0.0,1.0,WorldVision,406.0,...,4,0,13,8,0,1,2,0,0,negative
116413,931869631702331392,931881262201102336,928471317535514624,@WorldVision Arthur's Song Charity's is always...,14086764,9.318696e+17,0.0,1.0,WorldVision,177.0,...,1,0,21,10,0,1,0,0,0,positive
116414,1067115815001096193,1067207873061666821,196878226,@WorldVision 1 Peter 5:7 Cast all your anxiet...,14086764,1.067116e+18,0.0,1.0,WorldVision,12.0,...,1,0,13,6,0,1,0,0,0,positive


In [115]:
#save the data
df.to_csv("fix_labeled/extended_features.csv")

## Split the NGO data to ICRC related and others

In [4]:
#List of ICRC related NGOs
ICRC_list = ["RedCrossAU", "BritishRedCross", "philredcross", "redcrosscanada", "ICRC_Nairobi", "NepalRedCross", "RedCrossLebanon", "ICRC_library", "RedCross", "irishredcross", "IFRC_Europe"]

In [19]:
#save the data
df[df.relatedNGO.isin(ICRC_list)].to_csv("fix_labeled/ICRC_extended_features.csv")

,conversation_id,id,author_id,text,in_reply_to_user_id,replied_to,attachment,label,relatedNGO,author_followers,...,num_mentions,num_links,num_full_words,num_tokenized_words,num_hashtags,num_NGO_mentions,num_exclamation,num_question,has_question,sentiment
4987,1097533245275754497,1097540074479210498,821450289077227523,@RedCross Probably our best president ever \nFDR,6519522,1097533245275754497,0,0,RedCross,57,...,1,0,7,5,0,1,0,0,0,positive
4988,1097533245275754497,1097534331449470978,471235964,@RedCross 😍💪☝️💯 +goldenarm,6519522,1097533245275754497,0,0,RedCross,438,...,1,0,4,2,0,1,0,0,0,positive
4989,1025390517234933762,1026147716274024449,299694227,@RedCross You need to get ahead of the curve a...,6519522,1025390517234933762,0,0,RedCross,226,...,1,0,52,27,1,1,1,0,0,positive
4990,1025390517234933762,1025567647738548224,748970605073068032,@RedCross Praying for all of the families out ...,6519522,1025390517234933762,0,0,RedCross,7,...,1,0,13,5,0,1,0,0,0,neutral
4991,1025390517234933762,1025393326781071361,88666456,@RedCross I feel sorry for the wildlife being ...,6519522,1025390517234933762,0,0,RedCross,474,...,1,0,13,6,0,1,0,0,0,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111651,1021764082750418944,1021835636205137920,91125378,"@IFRC_Europe How can one donate to the relief,...",937963107456036865,1021764082750418944,0,1,IFRC_Europe,1443,...,1,0,12,6,0,1,0,1,1,neutral
111652,966005745669083141,966041767169740800,105835989,@IFRC_Europe @roteskreuzat @Federation @nicjon...,105835989,966041185931456513,0,1,IFRC_Europe,1670,...,4,0,10,4,0,1,0,0,0,negative
111653,942083045124919298,943488830589558785,740637548,@IFRC_Europe @eu_echo @HRC_SAMARITES Keep up w...,937963107456036865,942083045124919298,0,1,IFRC_Europe,5924,...,3,0,9,3,0,1,1,0,0,positive
111654,1330803185494192128,1331179030758961156,701322706495602688,@IFRC_Europe @BritishRedCross You bloody moron...,937963107456036865,1330803185494192128,0,1,IFRC_Europe,3473,...,2,0,49,23,0,1,1,0,0,negative


In [18]:
#save the data of additional NGOs
df[~df.relatedNGO.isin(ICRC_list)].to_csv("fix_labeled/OtherNGO_extended_features.csv")

,conversation_id,id,author_id,text,in_reply_to_user_id,replied_to,attachment,label,relatedNGO,author_followers,...,num_mentions,num_links,num_full_words,num_tokenized_words,num_hashtags,num_NGO_mentions,num_exclamation,num_question,has_question,sentiment
0,128892368731643905,129039727990026241,19279287,RT @Habitat_org For anyone in the Joplin area ...,33898911,128892368731643905,0,0,Habitat_org,1505,...,2,0,20,11,0,1,0,0,0,neutral
1,1513883133417979904,1514283971437383683,1461435318574493712,@Habitat_org @MrDrewScott @JonathanScott 👏👏👏👏👏,33898911,1513883133417979904,0,0,Habitat_org,1316,...,3,0,4,1,0,1,0,0,0,positive
2,1150423207129886720,1150806881885130752,889333376,@Habitat_org @HabitatPL 👍🙂,33898911,1150423207129886720,0,0,Habitat_org,69,...,2,0,3,1,0,1,0,0,0,positive
3,1150423207129886720,1150469798335266817,3293184712,@Habitat_org @HabitatPL So thankful for Habita...,33898911,1150423207129886720,0,0,Habitat_org,290,...,2,0,29,16,0,1,0,0,0,positive
4,1370041829408784384,1370457258673401860,3821162059,@Habitat_org Wow! This would be awesome for th...,33898911,1370041829408784384,0,0,Habitat_org,1142,...,1,0,18,9,0,1,1,0,0,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116411,1093570142888435713,1093570501375676416,474681580,"@WorldVision You go girl💃💃💃,there's alot of pe...",14086764,1093570142888435713,0,1,WorldVision,725,...,1,0,10,5,0,1,2,0,0,positive
116412,1062866805545492481,1063045758188969984,958755194212405248,@ericmetaxas @obianuju @WorldVision @PatriciaH...,105901883,1062866805545492481,0,1,WorldVision,406,...,4,0,13,8,0,1,2,0,0,negative
116413,931869631702331392,931881262201102336,928471317535514624,@WorldVision Arthur's Song Charity's is always...,14086764,931869631702331392,0,1,WorldVision,177,...,1,0,21,10,0,1,0,0,0,positive
116414,1067115815001096193,1067207873061666821,196878226,@WorldVision 1 Peter 5:7 Cast all your anxiet...,14086764,1067115815001096193,0,1,WorldVision,12,...,1,0,13,6,0,1,0,0,0,positive


## Topic modelling for tweets

In [ ]:
#read the data
df = pd.read_csv("fix_labeled/extended_features.csv", dtype="str", index_col=0)
df.head(5)

,conversation_id,id,author_id,text,in_reply_to_user_id,replied_to,attachment,label,relatedNGO,author_followers,...,num_tokenized_words,num_hashtags,num_NGO_mentions,num_exclamation,num_question,has_question,sentiment,retweet,topic,num_characters
0,128892368731643905,129039727990026241,19279287,RT @Habitat_org For anyone in the Joplin area ...,33898911,128892368731643905,0,0,Habitat_org,1505,...,11,0,1,0,0,0,neutral,1,4,137
1,1513883133417979904,1514283971437383683,1461435318574493712,@Habitat_org @MrDrewScott @JonathanScott 👏👏👏👏👏,33898911,1513883133417979904,0,0,Habitat_org,1316,...,1,0,1,0,0,0,positive,0,0,46
2,1150423207129886720,1150806881885130752,889333376,@Habitat_org @HabitatPL 👍🙂,33898911,1150423207129886720,0,0,Habitat_org,69,...,1,0,1,0,0,0,positive,0,3,26
3,1150423207129886720,1150469798335266817,3293184712,@Habitat_org @HabitatPL So thankful for Habita...,33898911,1150423207129886720,0,0,Habitat_org,290,...,16,0,1,0,0,0,positive,0,4,202
4,1370041829408784384,1370457258673401860,3821162059,@Habitat_org Wow! This would be awesome for th...,33898911,1370041829408784384,0,0,Habitat_org,1142,...,9,0,1,1,0,0,positive,0,5,114


In [ ]:
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'

#process the tweet text
def remove_links(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'bit.ly/\S+', '', text)
    text = text.strip('[link]')
    return text

def remove_users(text):
    text = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', text)
    text = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', text)
    return text

# further cleaning
def process_tweet(tweet, bigrams=False):
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = tweet.lower()
    #strip punctuation
    tweet = re.sub('['+my_punctuation + ']+', ' ', tweet)
    #remove double spacing
    tweet = re.sub('\s+', ' ', tweet)
    #remove numbers
    tweet = re.sub('([0-9]+)', '', tweet)
    #remove stopwords
    tweet_token_list = [word for word in tweet.split(' ')
                            if word not in my_stopwords]

    #apply word rooter
    tweet_token_list = [word_rooter(word) if '#' not in word else word
                        for word in tweet_token_list]

    tweet = ' '.join(tweet_token_list)
    return tweet

#do the processing for topic modeling prep
df['processed_tweet'] = df.text.apply(process_tweet)
df

<>:13: DeprecationWarning: invalid escape sequence \s
<>:25: DeprecationWarning: invalid escape sequence \s
<>:13: DeprecationWarning: invalid escape sequence \s
<>:25: DeprecationWarning: invalid escape sequence \s
/var/folders/z6/lf7x1vd54d39nq7yygwvfzcc0000gn/T/ipykernel_45550/2819116627.py:13: DeprecationWarning: invalid escape sequence \s
  text = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', text)
/var/folders/z6/lf7x1vd54d39nq7yygwvfzcc0000gn/T/ipykernel_45550/2819116627.py:25: DeprecationWarning: invalid escape sequence \s
  tweet = re.sub('\s+', ' ', tweet)


,conversation_id,id,author_id,text,in_reply_to_user_id,replied_to,attachment,label,relatedNGO,author_followers,...,num_hashtags,num_NGO_mentions,num_exclamation,num_question,has_question,sentiment,retweet,topic,num_characters,processed_tweet
0,128892368731643905,129039727990026241,19279287,RT @Habitat_org For anyone in the Joplin area ...,33898911,128892368731643905,0,0,Habitat_org,1505,...,0,1,0,0,0,neutral,1,4,137,anyon joplin area habitat human need volunt h...
1,1513883133417979904,1514283971437383683,1461435318574493712,@Habitat_org @MrDrewScott @JonathanScott 👏👏👏👏👏,33898911,1513883133417979904,0,0,Habitat_org,1316,...,0,1,0,0,0,positive,0,0,46,👏👏👏👏👏
2,1150423207129886720,1150806881885130752,889333376,@Habitat_org @HabitatPL 👍🙂,33898911,1150423207129886720,0,0,Habitat_org,69,...,0,1,0,0,0,positive,0,3,26,👍🙂
3,1150423207129886720,1150469798335266817,3293184712,@Habitat_org @HabitatPL So thankful for Habita...,33898911,1150423207129886720,0,0,Habitat_org,290,...,0,1,0,0,0,positive,0,4,202,thank habitat partner work elimin pipelin fos...
4,1370041829408784384,1370457258673401860,3821162059,@Habitat_org Wow! This would be awesome for th...,33898911,1370041829408784384,0,0,Habitat_org,1142,...,0,1,1,0,0,positive,0,5,114,wow would awesom us live multipl chemic sensi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116404,1093570142888435713,1093570501375676416,474681580,"@WorldVision You go girl💃💃💃,there's alot of pe...",14086764,1093570142888435713,0,1,WorldVision,725,...,0,1,2,0,0,positive,0,2,68,go girl💃💃💃 alot peopl root
116405,1062866805545492481,1063045758188969984,958755194212405248,@ericmetaxas @obianuju @WorldVision @PatriciaH...,105901883,1062866805545492481,0,1,WorldVision,406,...,0,1,2,0,0,negative,0,0,105,wow lost total respect world vision aw
116406,931869631702331392,931881262201102336,928471317535514624,@WorldVision Arthur's Song Charity's is always...,14086764,931869631702331392,0,1,WorldVision,177,...,0,1,0,0,0,positive,0,4,114,arthur song chariti alway world peopl great p...
116407,1067115815001096193,1067207873061666821,196878226,@WorldVision 1 Peter 5:7 Cast all your anxiet...,14086764,1067115815001096193,0,1,WorldVision,12,...,0,1,0,0,0,positive,0,4,79,peter cast anxieti care


In [ ]:
#assign vectorizer, which get rids of words that are more than in 90% of the tweets and those that are in less than 25.
vectorizer = CountVectorizer(max_df=0.9, min_df=25, token_pattern='\w+|\$[\d\.]+|\S+')

#apply transformation
tf = vectorizer.fit_transform(df['processed_tweet']).toarray()

#get feature names
tf_feature_names = vectorizer.get_feature_names_out()

#assign model
model = LatentDirichletAllocation(n_components=6, random_state=0)

<>:2: DeprecationWarning: invalid escape sequence \w
<>:2: DeprecationWarning: invalid escape sequence \w
/var/folders/z6/lf7x1vd54d39nq7yygwvfzcc0000gn/T/ipykernel_45550/3380264459.py:2: DeprecationWarning: invalid escape sequence \w
  vectorizer = CountVectorizer(max_df=0.9, min_df=25, token_pattern='\w+|\$[\d\.]+|\S+')


In [ ]:
#check that tf was done correctly in terms of tweet dimensions
tf.shape

(116409, 3743)

In [ ]:
#fit the model
model.fit(tf)

In [ ]:
#Display top n words from topics
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

display_topics(model, tf_feature_names, no_top_words=10)

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights,Topic 5 words,Topic 5 weights
0,scout,4307.1,red,2608.0,donat,8166.0,polic,18329.2,thank,4230.2,pleas,3655.8
1,boy,2574.1,cross,2470.2,money,3621.1,iranian,18236.2,god,3191.9,thank,3524.2
2,girl,2067.6,help,1996.3,blood,3382.0,use,11021.9,love,2985.7,amp,2777.7
3,church,2021.0,peopl,1411.9,red,3074.4,ambul,10438.2,work,2816.8,help,2043.0
4,’t,1770.2,po,1216.2,cross,2966.1,voic,9892.2,great,2813.7,refuge,1485.2
5,i,1532.0,war,948.2,peopl,2960.7,peopl,9831.2,help,2431.8,share,1386.1
6,presid,1509.2,kill,892.4,help,2811.5,protest,9674.2,good,2150.7,children,1321.1
7,’,1469.2,need,827.8,give,2468.8,transport,9542.2,peopl,1851.0,need,1226.2
8,trump,1409.2,الله,798.2,get,2034.0,station,8976.2,bless,1681.2,wait,1145.4
9,amp,1395.2,sa,718.2,go,1845.5,#مهسا,8972.2,need,1518.4,year,1007.5


In [ ]:
#assign topics to tweets
topic_probabilities = model.transform(tf)
topic_list = []
for i in topic_probabilities:
   topic_list.append(str(i.argmax()))

topic_df = pd.DataFrame(topic_list, columns=['topic'])

df["topic"] = topic_df["topic"]
df


,conversation_id,id,author_id,text,in_reply_to_user_id,replied_to,attachment,label,relatedNGO,author_followers,...,num_hashtags,num_NGO_mentions,num_exclamation,num_question,has_question,sentiment,retweet,topic,num_characters,processed_tweet
0,128892368731643905,129039727990026241,19279287,RT @Habitat_org For anyone in the Joplin area ...,33898911,128892368731643905,0,0,Habitat_org,1505,...,0,1,0,0,0,neutral,1,4,137,anyon joplin area habitat human need volunt h...
1,1513883133417979904,1514283971437383683,1461435318574493712,@Habitat_org @MrDrewScott @JonathanScott 👏👏👏👏👏,33898911,1513883133417979904,0,0,Habitat_org,1316,...,0,1,0,0,0,positive,0,0,46,👏👏👏👏👏
2,1150423207129886720,1150806881885130752,889333376,@Habitat_org @HabitatPL 👍🙂,33898911,1150423207129886720,0,0,Habitat_org,69,...,0,1,0,0,0,positive,0,3,26,👍🙂
3,1150423207129886720,1150469798335266817,3293184712,@Habitat_org @HabitatPL So thankful for Habita...,33898911,1150423207129886720,0,0,Habitat_org,290,...,0,1,0,0,0,positive,0,4,202,thank habitat partner work elimin pipelin fos...
4,1370041829408784384,1370457258673401860,3821162059,@Habitat_org Wow! This would be awesome for th...,33898911,1370041829408784384,0,0,Habitat_org,1142,...,0,1,1,0,0,positive,0,5,114,wow would awesom us live multipl chemic sensi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116404,1093570142888435713,1093570501375676416,474681580,"@WorldVision You go girl💃💃💃,there's alot of pe...",14086764,1093570142888435713,0,1,WorldVision,725,...,0,1,2,0,0,positive,0,2,68,go girl💃💃💃 alot peopl root
116405,1062866805545492481,1063045758188969984,958755194212405248,@ericmetaxas @obianuju @WorldVision @PatriciaH...,105901883,1062866805545492481,0,1,WorldVision,406,...,0,1,2,0,0,negative,0,0,105,wow lost total respect world vision aw
116406,931869631702331392,931881262201102336,928471317535514624,@WorldVision Arthur's Song Charity's is always...,14086764,931869631702331392,0,1,WorldVision,177,...,0,1,0,0,0,positive,0,4,114,arthur song chariti alway world peopl great p...
116407,1067115815001096193,1067207873061666821,196878226,@WorldVision 1 Peter 5:7 Cast all your anxiet...,14086764,1067115815001096193,0,1,WorldVision,12,...,0,1,0,0,0,positive,0,5,79,peter cast anxieti care


In [ ]:
#Drop processed tweet column
df = df.drop(["processed_tweet"], axis=1)

In [ ]:
df.to_csv("fix_labeled/extended_features.csv")